In [2]:
import requests
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

import numpy as np
import pandas as pd

In [35]:
Jeju = pd.read_csv('mini_project_1/Sejong.csv')
Jeju

,Unnamed: 0,업종,업소명,주요품목,가격,연락처,주소
0,0,한식_일반,색달가든 깐부고깃집,"오겹살, 목살(200g)",9000원,064-751-4884,제주도 제주시 제주특별자치도 제주시 오복3길 20
1,1,한식_일반,영락마을회관국수 연동본점,고기국수,8000원,010-9639-2265,제주도 제주시 제주특별자치도 제주시 신대로16길 37-1 (연동)
2,2,숙박업,제주스테이,스탠다드더블룸,38000원,064-755-7945,제주도 제주시 제주특별자치도 제주시 무근성7길 22(삼도이동)
3,3,한식_일반,장모식탁,전복돌솥밥+순두부찌개,14000원,064-805-7158,"제주도 제주시 제주특별자치도 제주시 연신로 51, 1층"
4,4,한식_일반,찍멍부멍,탕수육(미니),8000원,064-722-7090,"제주도 제주시 제주특별자치도 제주시 연신로 70, 103호"
...,...,...,...,...,...,...,...
296,296,한식_일반,다해,복해장국,9000원,064-749-9868,제주도 제주시 문송1길 17(연동)
297,297,한식_일반,내가찾은맛고을,삼계탕,10000원,064-702-0216,제주도 제주시 진남로8길 39(화북일동)
298,298,한식_육류,깜돈흑돼지,제주산 흑돼지(200g),20000원,064-712-9229,제주도 제주시 삼무로3길 6(연동)
299,299,한식_육류,길벗식당,오리구이(1마리),54000원,064-748-0848,제주도 제주시 월광로 162(노형동)


In [36]:
Jeju.drop('Unnamed: 0',axis = 1, inplace = True)

Jeju['주소_구,군'] = Jeju['주소'].apply(lambda x:x.split(' ')[0])

Jeju['주소_구,군'] = Jeju['주소'].apply(lambda x:x.replace('\xa0',' '))
Jeju['keyword'] = Jeju['주소_구,군']

Jeju_address_list = Jeju['keyword'].apply(lambda x:x.split(' '))

new_Jeju_address_list = []

for i,item in enumerate(Jeju_address_list):
    #print(ad_li[i][:2])
    address = item[0] + ' ' + item[1]
    new_Jeju_address_list.append(address)
    
new_Jeju_address_list

Jeju['keyword'] = new_Jeju_address_list

Jeju['keyword'] = Jeju['keyword'] + ' ' + Jeju['업소명']

In [37]:
# 카카오맵 url 가져오기
url = 'https://map.kakao.com/'

# 크롬 드라이버 실행하기
driver = webdriver.Chrome()

# 카카오맵 url로 이동하기
driver.get(url)

score_list = []

for key in Jeju['keyword'][:301]:
    # 검색창 클릭하기
    search = driver.find_element(By.CSS_SELECTOR, '#search\.keyword\.query')

    # keyword의 key를 검색창에 넣기
    search.send_keys(key)

    # 엔터키로 이동하기
    search.send_keys(Keys.RETURN)

    time.sleep(2)

    # 웹 스크립트 받아오기
    html = driver.page_source

    # 웹스크립트에서 원하는 콘텐츠 추출하기 - parsing
    soup = BeautifulSoup(html, 'html.parser')
    
    # keyword별 평점 가져오기
    try:
        # 평점 path 가져오기
        score_path = '#info\.search\.place\.list > li:nth-child(1) > div.rating.clickArea'
        
        # score_path를 split한 후 3번째 문자 추출하기
        score = soup.select(score_path)[0].text.split('\n')[3]
        
        # 평점을 빈 리스트 L에 추가하기
        score_list.append(score)
        
    except:
        score_list.append('별점 없음')
        
    search.clear()
    
score_list

Jeju['평점'] = score_list
Jeju.to_csv('Jejuscore.csv', encoding='utf-8-sig')

<>:14: SyntaxWarning: invalid escape sequence '\.'
<>:33: SyntaxWarning: invalid escape sequence '\.'
<>:14: SyntaxWarning: invalid escape sequence '\.'
<>:33: SyntaxWarning: invalid escape sequence '\.'
C:\Users\vvv\AppData\Local\Temp\ipykernel_4508\872744076.py:14: SyntaxWarning: invalid escape sequence '\.'
  search = driver.find_element(By.CSS_SELECTOR, '#search\.keyword\.query')
C:\Users\vvv\AppData\Local\Temp\ipykernel_4508\872744076.py:33: SyntaxWarning: invalid escape sequence '\.'
  score_path = '#info\.search\.place\.list > li:nth-child(1) > div.rating.clickArea'
